In [1]:
# Jupyter Cell 1: Setup and Imports
import mysql.connector
import pandas as pd

# Connect to MySQL and create DB & table
def init_db():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",  # Change if needed
    )
    cursor = conn.cursor()
    cursor.execute("CREATE DATABASE IF NOT EXISTS ACT001")
    conn.database = "ACT001"
    cursor.execute("""
        CREATE TABLE IF NOT EXISTS students (
            id INT AUTO_INCREMENT PRIMARY KEY,
            name VARCHAR(100),
            age INT,
            grade VARCHAR(10)
        )
    """)
    conn.commit()
    cursor.close()
    conn.close()
    print("Database and table ready.")

In [2]:
# Jupyter Cell 2: Insert from CSV
def insert_from_csv(csv_path="users.csv"):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ACT001"
    )
    cursor = conn.cursor()

    try:
        df = pd.read_csv(csv_path)
        df.columns = df.columns.str.strip().str.lower()
        for _, row in df.iterrows():
            cursor.execute("INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)",
                           (row['name'], int(row['age']), row['grade']))
        conn.commit()
        print("Data inserted from CSV.")
    except FileNotFoundError:
        print("CSV file not found.")
    except Exception as e:
        print(f"Error: {e}")
    finally:
        cursor.close()
        conn.close()


In [3]:
# Jupyter Cell 3: Insert manually
def insert_manual(name, age, grade):
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ACT001"
    )
    cursor = conn.cursor()
    cursor.execute("INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)", (name, age, grade))
    conn.commit()
    cursor.close()
    conn.close()
    print("Manual data inserted.")

In [4]:
# Jupyter Cell 4: View data
def view_students():
    conn = mysql.connector.connect(
        host="localhost",
        user="root",
        password="",
        database="ACT001"
    )
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM students")
    for row in cursor.fetchall():
        print(row)
    cursor.close()
    conn.close()


# Jupyter Cell 5: Flask Web UI (run in terminal or another environment, not in Jupyter)
# Save as app.py and run using: flask run
from flask import Flask, request, render_template_string

app = Flask(__name__)

html_form = '''
<!doctype html>
<title>Add Student</title>
<h2>Enter Student Info</h2>
<form method=post>
  Name: <input type=text name=name><br>
  Age: <input type=number name=age><br>
  Grade: <input type=text name=grade><br>
  <input type=submit value=Submit>
</form>
<hr>
<h3>Students:</h3>
<ul>
  {% for s in students %}
    <li>{{s[1]}} - {{s[2]}} years - Grade {{s[3]}}</li>
  {% endfor %}
</ul>
'''

@app.route('/', methods=['GET', 'POST'])
def home():
    conn = mysql.connector.connect(host="localhost", user="root", password="", database="ACT001")
    cursor = conn.cursor()
    if request.method == 'POST':
        name = request.form['name']
        age = int(request.form['age'])
        grade = request.form['grade']
        cursor.execute("INSERT INTO students (name, age, grade) VALUES (%s, %s, %s)", (name, age, grade))
        conn.commit()
    cursor.execute("SELECT * FROM students")
    students = cursor.fetchall()
    cursor.close()
    conn.close()
    return render_template_string(html_form, students=students)

if __name__ == '__main__':
    app.run(debug=True)


In [5]:
# Jupyter Cell 6: Run init
init_db()


Database and table ready.


In [6]:
from flask import Flask, render_template, request, redirect
import mysql.connector


In [7]:
!where python


C:\Program Files\Python313\python.exe
C:\Users\rajuk\AppData\Local\Programs\Python\Python314\python.exe
C:\Users\rajuk\AppData\Local\Microsoft\WindowsApps\python.exe


In [8]:
import sys
print("Python being used:", sys.executable)


Python being used: C:\Program Files\Python313\python.exe
